# Decorator Pattern Tutorial 🎨

## What is the Decorator Pattern?

The Decorator Pattern allows you to add new functionality to objects dynamically without altering their structure. It's like adding toppings to a pizza - you start with a basic pizza and keep adding layers of toppings, each one enhancing the pizza without changing what a pizza fundamentally is.

**Real-world analogy**: Think of dressing for cold weather. You start with a shirt, add a sweater, then a jacket. Each layer adds functionality (warmth) without changing the fact that you're still wearing clothes.

## Table of Contents
1. [What is the Decorator Pattern?](#what-is-the-decorator-pattern)
2. [Why Do We Need It?](#why-do-we-need-it)
3. [Simple Implementation](#simple-implementation)
4. [Understanding the Implementation](#understanding-the-implementation)
5. [Python's Built-in Decorators (Advanced)](#pythons-built-in-decorators)
6. [When to Use (and When NOT to Use)](#when-to-use-and-when-not-to-use)

## Why Do We Need It?

Let's see what happens when we try to add features without the decorator pattern:

In [ ]:
# Problem: Class explosion when trying to add multiple features

class Coffee:
    def cost(self):
        return 5.0
    
    def description(self):
        return "Simple coffee"

class CoffeeWithMilk:
    def cost(self):
        return 5.0 + 1.0
    
    def description(self):
        return "Coffee with milk"

class CoffeeWithMilkAndSugar:
    def cost(self):
        return 5.0 + 1.0 + 0.5
    
    def description(self):
        return "Coffee with milk and sugar"

class CoffeeWithMilkAndSugarAndWhip:
    def cost(self):
        return 5.0 + 1.0 + 0.5 + 1.5
    
    def description(self):
        return "Coffee with milk, sugar, and whip"

# Problem: We need a new class for every combination!
# What about coffee with just sugar? Or just whip? Or milk and whip but no sugar?
# This leads to an explosion of classes!

## Simple Implementation

Let's solve this with the Decorator Pattern:

In [ ]:
from abc import ABC, abstractmethod

# Component interface
class Beverage(ABC):
    @abstractmethod
    def cost(self):
        pass
    
    @abstractmethod
    def description(self):
        pass

# Concrete component
class Coffee(Beverage):
    def cost(self):
        return 5.0
    
    def description(self):
        return "Simple coffee"

# Base decorator
class CondimentDecorator(Beverage):
    def __init__(self, beverage: Beverage):
        self._beverage = beverage

# Concrete decorators
class Milk(CondimentDecorator):
    def cost(self):
        return self._beverage.cost() + 1.0
    
    def description(self):
        return self._beverage.description() + ", milk"

class Sugar(CondimentDecorator):
    def cost(self):
        return self._beverage.cost() + 0.5
    
    def description(self):
        return self._beverage.description() + ", sugar"

class WhipCream(CondimentDecorator):
    def cost(self):
        return self._beverage.cost() + 1.5
    
    def description(self):
        return self._beverage.description() + ", whip cream"

In [ ]:
# Now we can create any combination dynamically!

# Simple coffee
my_coffee = Coffee()
print(f"{my_coffee.description()}: ${my_coffee.cost():.2f}")

# Coffee with milk and sugar
my_coffee = Coffee()
my_coffee = Milk(my_coffee)
my_coffee = Sugar(my_coffee)
print(f"{my_coffee.description()}: ${my_coffee.cost():.2f}")

# Coffee with double milk and whip
my_coffee = Coffee()
my_coffee = Milk(my_coffee)
my_coffee = Milk(my_coffee)  # Double milk!
my_coffee = WhipCream(my_coffee)
print(f"{my_coffee.description()}: ${my_coffee.cost():.2f}")

## Understanding the Implementation

### Key Concepts:

1. **Component Interface**: Defines the interface for objects that can have responsibilities added
2. **Concrete Component**: The basic object to which we add functionality
3. **Decorator**: Maintains a reference to a Component and defines an interface conforming to Component
4. **Concrete Decorators**: Add responsibilities to the component

### How it works:
- Each decorator "wraps" the object it's decorating
- When you call a method, it adds its own behavior then delegates to the wrapped object
- You can wrap decorators with other decorators, creating a chain

## Python's Built-in Decorators

Python has built-in support for decorators using the `@` syntax. While these are function/method decorators (not the same as the object-oriented Decorator Pattern), they follow similar principles:

In [ ]:
import time
import functools

# Function decorator for timing
def timer(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"{func.__name__} took {end - start:.4f} seconds")
        return result
    return wrapper

# Function decorator for logging
def logger(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print(f"Calling {func.__name__} with args={args}, kwargs={kwargs}")
        result = func(*args, **kwargs)
        print(f"{func.__name__} returned {result}")
        return result
    return wrapper

# Using decorators
@timer
@logger
def calculate_sum(n):
    return sum(range(n))

result = calculate_sum(1000000)

In [ ]:
# Creating a class-based decorator for validation
class ValidateInput:
    def __init__(self, min_value=None, max_value=None):
        self.min_value = min_value
        self.max_value = max_value
    
    def __call__(self, func):
        @functools.wraps(func)
        def wrapper(instance, value):
            if self.min_value is not None and value < self.min_value:
                raise ValueError(f"Value {value} is less than minimum {self.min_value}")
            if self.max_value is not None and value > self.max_value:
                raise ValueError(f"Value {value} is greater than maximum {self.max_value}")
            return func(instance, value)
        return wrapper

class Temperature:
    def __init__(self):
        self._celsius = 0
    
    @ValidateInput(min_value=-273.15, max_value=1000)
    def set_celsius(self, value):
        self._celsius = value
        print(f"Temperature set to {value}°C")

# Test the validation
temp = Temperature()
temp.set_celsius(25)  # OK

try:
    temp.set_celsius(-300)  # Will raise an error
except ValueError as e:
    print(f"Error: {e}")

## Advanced Example: Text Processing Pipeline

Let's create a more complex example showing how decorators can build a text processing pipeline:

In [ ]:
from abc import ABC, abstractmethod
import re

# Component
class TextProcessor(ABC):
    @abstractmethod
    def process(self, text: str) -> str:
        pass

# Concrete component
class PlainText(TextProcessor):
    def process(self, text: str) -> str:
        return text

# Base decorator
class TextDecorator(TextProcessor):
    def __init__(self, processor: TextProcessor):
        self._processor = processor

# Concrete decorators
class UppercaseDecorator(TextDecorator):
    def process(self, text: str) -> str:
        return self._processor.process(text).upper()

class TrimDecorator(TextDecorator):
    def process(self, text: str) -> str:
        return self._processor.process(text).strip()

class RemoveExtraSpacesDecorator(TextDecorator):
    def process(self, text: str) -> str:
        processed = self._processor.process(text)
        return re.sub(r'\s+', ' ', processed)

class CensorDecorator(TextDecorator):
    def __init__(self, processor: TextProcessor, words_to_censor: list):
        super().__init__(processor)
        self.words_to_censor = words_to_censor
    
    def process(self, text: str) -> str:
        processed = self._processor.process(text)
        for word in self.words_to_censor:
            processed = processed.replace(word, '*' * len(word))
        return processed

# Building a text processing pipeline
text = "  Hello   world!  This is a   BAD  example.  "

# Create a processing pipeline
processor = PlainText()
processor = TrimDecorator(processor)
processor = RemoveExtraSpacesDecorator(processor)
processor = CensorDecorator(processor, ['BAD', 'bad'])
processor = UppercaseDecorator(processor)

result = processor.process(text)
print(f"Original: '{text}'")
print(f"Processed: '{result}'")

## When to Use (and When NOT to Use)

### Use Decorator Pattern when:
- You want to add responsibilities to objects dynamically and transparently
- You want to add responsibilities that can be withdrawn
- Extension by subclassing is impractical (would lead to explosion of subclasses)
- You need to combine multiple behaviors in various ways

### Don't use when:
- The component interface is too complex (decorators need to implement all methods)
- You only need a fixed set of combinations (regular inheritance might be simpler)
- Performance is critical (each decorator adds a layer of indirection)

### Real-world applications:
- Java I/O streams (BufferedInputStream, DataInputStream, etc.)
- Web frameworks middleware (authentication, logging, compression)
- GUI components (scrollbars, borders, etc.)
- Game development (power-ups, equipment effects)